# 🚢 Port-to-Rail Logistics: GPU-Accelerated EDA
## DGX Spark Frontier Hackathon - December 2025

This notebook uses **RAPIDS cuDF** for GPU-accelerated data processing.

**Expected Speedups:**
- Data loading: ~10-20x faster
- GroupBy operations: ~20-50x faster
- Rolling windows: ~30x faster

In [1]:
# Check GPU availability
!nvidia-smi

Sat Dec 13 09:48:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   35C    P0             56W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
# Imports
import time
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# GPU imports
try:
    import cudf
    import cupy as cp
    GPU_AVAILABLE = True
    print(f"✅ RAPIDS cuDF version: {cudf.__version__}")
    print(f"✅ CuPy version: {cp.__version__}")
except ImportError:
    print("⚠️ RAPIDS not available, falling back to pandas")
    import pandas as cudf  # Fallback
    import numpy as cp
    GPU_AVAILABLE = False

import numpy as np
import pandas as pd

# Paths
DATA_DIR = Path("data")
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"\n📁 Data directory: {DATA_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")

✅ RAPIDS cuDF version: 25.10.00
✅ CuPy version: 13.6.0

📁 Data directory: data
📁 Output directory: output


---
## 1. Load Port Activity Data (GPU)

In [40]:
t0 = time.time()

port_file = "Daily_Port_Activity_Data_and_Trade_Estimates.csv"

df_all = cudf.read_csv(port_file)

load_time = time.time() - t0
print(f"\nLoaded {len(df_all):,} records in {load_time:.2f}s")
print(f"Columns: {list(df_all.columns)}")


Loaded 5,010,140 records in 0.31s
Columns: ['date', 'year', 'month', 'day', 'portid', 'portname', 'country', 'ISO3', 'portcalls_container', 'portcalls_dry_bulk', 'portcalls_general_cargo', 'portcalls_roro', 'portcalls_tanker', 'portcalls_cargo', 'portcalls', 'import_container', 'import_dry_bulk', 'import_general_cargo', 'import_roro', 'import_tanker', 'import_cargo', 'import', 'export_container', 'export_dry_bulk', 'export_general_cargo', 'export_roro', 'export_tanker', 'export_cargo', 'export', 'ObjectId']


In [41]:
print("Data Types:")
print(df_all.dtypes)
print(f"\nMemory Usage: {df_all.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

Data Types:
date                        object
year                         int64
month                        int64
day                          int64
portid                      object
portname                    object
country                     object
ISO3                        object
portcalls_container          int64
portcalls_dry_bulk           int64
portcalls_general_cargo      int64
portcalls_roro               int64
portcalls_tanker             int64
portcalls_cargo              int64
portcalls                    int64
import_container           float64
import_dry_bulk            float64
import_general_cargo       float64
import_roro                float64
import_tanker              float64
import_cargo               float64
import                     float64
export_container           float64
export_dry_bulk            float64
export_general_cargo       float64
export_roro                float64
export_tanker              float64
export_cargo               float64
export  

In [42]:
print("Filtering to United States...")
t0 = time.time()

df_us = df_all[df_all['country'] == 'United States'].copy()

print(f"US records: {len(df_us):,} ({len(df_us)/len(df_all)*100:.1f}% of total)")
print(f"Filter time: {time.time()-t0:.3f}s")
print(f"US ports: {df_us['portname'].nunique()}")

del df_all
if GPU_AVAILABLE:
    import gc
    gc.collect()
    cp.get_default_memory_pool().free_all_blocks()

Filtering to United States...
US records: 287,736 (5.7% of total)
Filter time: 0.017s
US ports: 114


In [47]:
df_us['date'] = cudf.to_datetime(df_us['date'])

---
## 2. Top US Ports Analysis

In [49]:
print("Computing port rankings...")
t0 = time.time()

port_stats = df_us.groupby('portname').agg({
    'portcalls': 'sum',
    'portcalls_container': 'sum',
    'portcalls_tanker': 'sum',
    'portcalls_dry_bulk': 'sum',
    'import_cargo': 'sum',
    'export_cargo': 'sum',
}).reset_index()

# total volume = import + export cargo
port_stats['total_volume'] = port_stats['import_cargo'] + port_stats['export_cargo']
port_stats['container_ratio'] = (port_stats['portcalls_container'] / port_stats['portcalls'] * 100).fillna(0)

port_stats = port_stats.sort_values('portcalls', ascending=False)

print(f"Aggregation time: {time.time()-t0:.3f}s")
print("\n TOP 20 US PORTS:")

port_stats_pd = port_stats.to_pandas() if GPU_AVAILABLE else port_stats
display_cols = ['portname', 'portcalls', 'portcalls_container', 'container_ratio', 'total_volume']
print(port_stats_pd.head(20)[display_cols].to_string(index=False))

Computing port rankings...
Aggregation time: 0.011s

 TOP 20 US PORTS:
              portname  portcalls  portcalls_container  container_ratio  total_volume
               Houston      50645                 6797        13.420871  3.857563e+08
           New Orleans      35207                 2957         8.398898  4.932603e+08
   New York-New Jersey      26714                15114        56.577076  5.079804e+08
Los Angeles-Long Beach      26176                12990        49.625611  1.089067e+09
       South Louisiana      16826                   29         0.172352  3.475274e+08
       Port Everglades      15927                10115        63.508508  3.728503e+07
              Savannah      15717                11062        70.382388  3.655677e+08
             Baltimore      12352                 3442        27.865933  1.726050e+08
            Charleston      12100                 8364        69.123967  1.847994e+08
        Corpus Christi      11842                   26         0.2195

In [50]:
port_stats_pd['port_type'] = port_stats_pd['container_ratio'].apply(
    lambda x: 'Container Hub' if x > 60 else ('Bulk Terminal' if x < 20 else 'Mixed')
)

print("\nPORT TYPE DISTRIBUTION:")
print(port_stats_pd['port_type'].value_counts())

print("\nTOP 10 CONTAINER PORTS:")
container_ports = port_stats_pd.nlargest(10, 'portcalls_container')[['portname', 'portcalls_container', 'container_ratio']]
print(container_ports.to_string(index=False))


PORT TYPE DISTRIBUTION:
port_type
Bulk Terminal    81
Mixed            19
Container Hub    14
Name: count, dtype: int64

TOP 10 CONTAINER PORTS:
              portname  portcalls_container  container_ratio
   New York-New Jersey                15114        56.577076
Los Angeles-Long Beach                12990        49.625611
              Savannah                11062        70.382388
       Port Everglades                10115        63.508508
            Charleston                 8364        69.123967
      Port of Virginia                 7797        88.562017
               Oakland                 7032        95.777717
               Houston                 6797        13.420871
                 Miami                 5522        65.683359
               Norfolk                 4625        57.668329


---
## 3. GPU-Accelerated Feature Engineering

In [56]:
TOP_N = 30
top_ports = port_stats_pd.head(TOP_N)['portname'].tolist()

print(f"Focusing on top {TOP_N} ports...")
df = df_us[df_us['portname'].isin(top_ports)].copy()
print(f"Records: {len(df):,}")

print(df.head()['portname'])
print("\n --------------------------------")
print(df.tail()['portname'])

df = df.sort_values(['portname', 'date'])

Focusing on top 30 ports...
Records: 75,720
280004    New Orleans
280005    New Orleans
280052    New Orleans
280056    New Orleans
280060    New Orleans
Name: portname, dtype: object

 --------------------------------
4134395    Marcus Hook
4134396    Marcus Hook
4134397    Marcus Hook
4134398    Marcus Hook
4134399    Marcus Hook
Name: portname, dtype: object


In [57]:
# TEMPORAL FEATURES (GPU-accelerated)
print("Adding temporal features...")
t0 = time.time()

df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_year'] = df['date'].dt.dayofyear
df['week_of_year'] = df['date'].dt.isocalendar().week.astype('int32')
df['is_weekend'] = (df['day_of_week'] >= 5).astype('int8')
df['is_month_end'] = df['date'].dt.is_month_end.astype('int8')

if GPU_AVAILABLE:
    df['month_sin'] = cp.sin(2 * cp.pi * df['month'].values / 12)
    df['month_cos'] = cp.cos(2 * cp.pi * df['month'].values / 12)
    df['dow_sin'] = cp.sin(2 * cp.pi * df['day_of_week'].values / 7)
    df['dow_cos'] = cp.cos(2 * cp.pi * df['day_of_week'].values / 7)
else:
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

print(f"⏱️ Temporal features: {time.time()-t0:.3f}s")

Adding temporal features...
⏱️ Temporal features: 0.017s


In [58]:
print("Adding cargo features...")
t0 = time.time()

# Safe division helper
def safe_div(a, b, fill=0):
    result = a / b
    result = result.fillna(fill)
    if GPU_AVAILABLE:
        result = result.replace([cp.inf, -cp.inf], fill)
    else:
        result = result.replace([np.inf, -np.inf], fill)
    return result

df['container_pct'] = safe_div(df['portcalls_container'], df['portcalls_cargo']) * 100
df['tanker_pct'] = safe_div(df['portcalls_tanker'], df['portcalls_cargo']) * 100
df['bulk_pct'] = safe_div(df['portcalls_dry_bulk'], df['portcalls_cargo']) * 100

df['import_export_ratio'] = safe_div(df['import_cargo'], df['export_cargo'], fill=1)
df['trade_balance'] = df['export_cargo'] - df['import_cargo']
df['total_volume'] = df['import_cargo'] + df['export_cargo']
df['volume_per_call'] = safe_div(df['total_volume'], df['portcalls_cargo'])

print(f"Cargo features: {time.time()-t0:.3f}s")

Adding cargo features...
Cargo features: 0.048s


In [62]:
print("Computing rolling statistics...")
t0 = time.time()

# cuDF doesn't support groupby().transform() with rolling
# Instead, compute per-port using a loop or apply pattern

if GPU_AVAILABLE:
    # Method: Sort by port+date, compute rolling, then handle port boundaries
    df = df.sort_values(['portname', 'date']).reset_index(drop=True)

    # For each window, compute rolling stats per port
    for window in [7, 14, 30]:
        # Use groupby + rolling (cuDF 23.10+ syntax)
        df[f'ma{window}'] = df.groupby('portname')['portcalls'].rolling(window, min_periods=1).mean().reset_index(drop=True)
        df[f'std{window}'] = df.groupby('portname')['portcalls'].rolling(window, min_periods=1).std().reset_index(drop=True).fillna(0)

    # Import rolling
    df['import_ma7'] = df.groupby('portname')['import_cargo'].rolling(7, min_periods=1).mean().reset_index(drop=True)
else:
    # Pandas version
    for window in [7, 14, 30]:
        df[f'ma{window}'] = df.groupby('portname')['portcalls'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
        df[f'std{window}'] = df.groupby('portnaame')['portcalls'].transform(
            lambda x: x.rolling(window, min_periods=1).std()
        ).fillna(0)

    df['import_ma7'] = df.groupby('portname')['import_cargo'].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )

print(f"⏱️ Rolling stats: {time.time()-t0:.3f}s")

Computing rolling statistics...
⏱️ Rolling stats: 0.072s


In [63]:
# Z-SCORES & SURGE DETECTION
print("Computing z-scores and surge detection...")
t0 = time.time()

# Z-scores
df['zscore_7d'] = safe_div(df['portcalls'] - df['ma7'], df['std7'].replace(0, 1))
df['zscore_30d'] = safe_div(df['portcalls'] - df['ma30'], df['std30'].replace(0, 1))

# Surge flags
df['surge_2std'] = (df['zscore_7d'] > 2).astype('int8')
df['surge_3std'] = (df['zscore_7d'] > 3).astype('int8')

# Volume surge (50% above 7-day avg)
df['volume_surge'] = (df['import_cargo'] > df['import_ma7'] * 1.5).astype('int8')

print(f"⏱️ Surge detection: {time.time()-t0:.3f}s")

Computing z-scores and surge detection...
⏱️ Surge detection: 0.029s


In [64]:
# LAG FEATURES (for prediction)
print("Adding lag features...")
t0 = time.time()

for lag in [1, 3, 7, 14]:
    df[f'calls_lag{lag}'] = df.groupby('portname')['portcalls'].shift(lag)
    df[f'import_lag{lag}'] = df.groupby('portname')['import_cargo'].shift(lag)

# Momentum
df['momentum_3d'] = df.groupby('portname')['portcalls'].diff(3)
df['momentum_7d'] = df.groupby('portname')['portcalls'].diff(7)
df['pct_change'] = df.groupby('portname')['portcalls'].pct_change()

print(f"⏱️ Lag features: {time.time()-t0:.3f}s")

Adding lag features...
⏱️ Lag features: 0.077s


In [65]:
# PORT-LEVEL STATIC FEATURES
print("Adding port-level features...")
t0 = time.time()

# Activity rank - compute differently for cuDF
if GPU_AVAILABLE:
    # Calculate mean portcalls per port
    port_avg = df.groupby('portname')['portcalls'].mean().reset_index()
    port_avg.columns = ['portname', 'avg_portcalls']
    port_avg['activity_rank'] = port_avg['avg_portcalls'].rank(ascending=False)

    # Merge back to main df
    df = df.merge(port_avg[['portname', 'activity_rank']], on='portname', how='left')

    # Port tier - use merge instead of map
    def get_tier(rank):
        if rank <= 10: return 'mega'
        elif rank <= 20: return 'major'
        else: return 'medium'

    # Create tier mapping as a dataframe
    port_avg['port_tier'] = port_avg['activity_rank'].to_pandas().apply(get_tier)
    tier_df = port_avg[['portname', 'port_tier']].copy()

    # Convert tier_df to cudf if needed
    if not isinstance(tier_df, cudf.DataFrame):
        tier_df = cudf.DataFrame(tier_df)

    df = df.merge(tier_df, on='portname', how='left')
else:
    # Pandas version
    port_rank = df.groupby('portname')['portcalls'].mean().rank(ascending=False)
    df['activity_rank'] = df['portname'].map(port_rank)

    def get_tier(rank):
        if rank <= 10: return 'mega'
        elif rank <= 20: return 'major'
        else: return 'medium'

    tier_map = {port: get_tier(rank) for port, rank in port_rank.items()}
    df['port_tier'] = df['portname'].map(tier_map)

print(f"⏱️ Port features: {time.time()-t0:.3f}s")

Adding port-level features...
⏱️ Port features: 0.029s


In [66]:
# PREDICTION TARGETS
print("Creating prediction targets...")
t0 = time.time()

# Next-day forecasts (24h)
df['target_calls_1d'] = df.groupby('portname')['portcalls'].shift(-1)
df['target_import_1d'] = df.groupby('portname')['import_cargo'].shift(-1)
df['target_surge_1d'] = df.groupby('portname')['surge_2std'].shift(-1)

# 3-day forecasts (72h)
df['target_calls_3d'] = df.groupby('portname')['portcalls'].shift(-3)

# 7-day forecasts
df['target_calls_7d'] = df.groupby('portname')['portcalls'].shift(-7)

print(f"⏱️ Targets created: {time.time()-t0:.3f}s")
print(f"\n📊 Total features: {len(df.columns)}")

Creating prediction targets...
⏱️ Targets created: 0.039s

📊 Total features: 76


---
## 5. 📈 Surge Analysis

In [67]:
# SURGE STATISTICS
print("Computing surge statistics...")
t0 = time.time()

if GPU_AVAILABLE:
    # cuDF-compatible aggregation
    surge_stats = df.groupby('portname').agg({
        'surge_2std': 'sum',
        'surge_3std': 'sum',
        'volume_surge': 'sum',
        'portcalls': ['mean', 'std', 'max'],
        'zscore_7d': 'max',
        'import_cargo': 'mean'
    }).reset_index()

    # Flatten column names
    surge_stats.columns = ['portname', 'surge_2std_days', 'surge_3std_days', 'volume_surge_days',
                           'avg_calls', 'std_calls', 'max_calls', 'max_zscore', 'avg_import']

    # Get total days per port using merge instead of map
    days_per_port = df.groupby('portname').size().reset_index()
    days_per_port.columns = ['portname', 'total_days']

    surge_stats = surge_stats.merge(days_per_port, on='portname', how='left')
    surge_stats['surge_rate'] = surge_stats['surge_2std_days'] / surge_stats['total_days'] * 100

    # Sort by surge count
    surge_stats = surge_stats.sort_values('surge_2std_days', ascending=False)

    # Convert to pandas for display
    surge_stats_pd = surge_stats.to_pandas()
else:
    surge_stats = df.groupby('portname').agg({
        'surge_2std': 'sum',
        'surge_3std': 'sum',
        'volume_surge': 'sum',
        'portcalls': ['mean', 'std', 'max'],
        'zscore_7d': 'max',
        'import_cargo': 'mean'
    }).reset_index()

    surge_stats.columns = ['portname', 'surge_2std_days', 'surge_3std_days', 'volume_surge_days',
                           'avg_calls', 'std_calls', 'max_calls', 'max_zscore', 'avg_import']

    days_per_port = df.groupby('portname').size()
    surge_stats['total_days'] = surge_stats['portname'].map(days_per_port)
    surge_stats['surge_rate'] = surge_stats['surge_2std_days'] / surge_stats['total_days'] * 100

    surge_stats = surge_stats.sort_values('surge_2std_days', ascending=False)
    surge_stats_pd = surge_stats

print(f"⏱️ Surge analysis: {time.time()-t0:.3f}s")

print("\n🚨 TOP 15 SURGE-PRONE PORTS:")
print(surge_stats_pd.head(15)[['portname', 'surge_2std_days', 'surge_rate', 'max_zscore', 'avg_calls']].to_string(index=False))

Computing surge statistics...
⏱️ Surge analysis: 0.018s

🚨 TOP 15 SURGE-PRONE PORTS:
        portname  surge_2std_days  surge_rate  max_zscore  avg_calls
          Duluth               41    1.624406    2.267787   1.708003
  Wilmington, DE               29    1.148970    2.267787   2.223851
     Port Arthur               26    1.030111    2.267787   2.912441
     New Orleans               23    0.911252    2.234217  13.948891
Port of Virginia               21    0.832013    2.206837   3.488114
    Lake Charles               21    0.832013    2.267787   2.143027
          Tacoma               21    0.832013    2.267787   2.378368
      Texas City               20    0.792393    2.267787   1.981379
           Miami               20    0.792393    2.267787   3.330824
          Mobile               20    0.792393    2.225052   3.146593
       Galveston               18    0.713154    2.165251   1.684628
     Marcus Hook               18    0.713154    2.165251   1.814976
    Jacksonville  

In [68]:
# Overall surge statistics
total_surge_2std = surge_stats_pd['surge_2std_days'].sum()
total_volume_surge = surge_stats_pd['volume_surge_days'].sum()
avg_surge_rate = surge_stats_pd['surge_rate'].mean()

print("\n📊 SURGE SUMMARY:")
print(f"  Total surge events (2σ): {total_surge_2std:,}")
print(f"  Total volume surge events: {total_volume_surge:,}")
print(f"  Average surge rate: {avg_surge_rate:.2f}%")


📊 SURGE SUMMARY:
  Total surge events (2σ): 523
  Total volume surge events: 16,111
  Average surge rate: 0.69%


---
## 6. 🌊 Load Additional Data (Chokepoints, Disruptions)

In [69]:
# Load chokepoint data
print("Loading chokepoint data...")
t0 = time.time()

chokepoint_file = "Daily_Chokepoints_Data.csv"
df_choke = cudf.read_csv(chokepoint_file)

print(f"✅ Loaded {len(df_choke):,} chokepoint records in {time.time()-t0:.3f}s")
print(f"🌊 Chokepoints: {df_choke['portname'].unique().to_pandas().tolist() if GPU_AVAILABLE else df_choke['portname'].unique().tolist()}")

Loading chokepoint data...
✅ Loaded 70,728 chokepoint records in 0.016s
🌊 Chokepoints: ['Suez Canal', 'Panama Canal', 'Bosporus Strait', 'Bab el-Mandeb Strait', 'Malacca Strait', 'Strait of Hormuz', 'Cape of Good Hope', 'Gibraltar Strait', 'Dover Strait', 'Oresund Strait', 'Taiwan Strait', 'Korea Strait', 'Tsugaru Strait', 'Luzon Strait', 'Lombok Strait', 'Ombai Strait', 'Bohai Strait', 'Torres Strait', 'Sunda Strait', 'Makassar Strait', 'Magellan Strait', 'Yucatan Channel', 'Windward Passage', 'Mona Passage', 'Balabac Strait', 'Bering Strait', 'Mindoro Strait', 'Kerch Strait']


In [70]:
# Chokepoint statistics
choke_stats = df_choke.groupby('portname').agg({
    'n_cargo': ['mean', 'std', 'max'],
    'n_container': ['mean', 'max'],
    'capacity_cargo': ['mean', 'max']
}).reset_index()

choke_stats.columns = ['_'.join(col).strip('_') for col in choke_stats.columns]
choke_stats = choke_stats.sort_values('n_cargo_mean', ascending=False)

choke_pd = choke_stats.to_pandas() if GPU_AVAILABLE else choke_stats
print("\n🌊 CHOKEPOINT ACTIVITY (by avg daily cargo ships):")
print(choke_pd[['portname', 'n_cargo_mean', 'n_cargo_max', 'n_container_mean']].head(15).to_string(index=False))


🌊 CHOKEPOINT ACTIVITY (by avg daily cargo ships):
            portname  n_cargo_mean  n_cargo_max  n_container_mean
       Taiwan Strait    186.418052          345         85.375297
        Korea Strait    162.794141          314         56.684481
        Bohai Strait    139.294537          347         21.382423
      Malacca Strait    120.007126          198         49.560966
        Dover Strait    108.002375          173         34.317894
    Gibraltar Strait     87.964766          127         32.038401
     Bosporus Strait     72.866587          126          8.236738
     Makassar Strait     47.211006           77          1.981394
        Luzon Strait     46.079177          160          6.667458
   Cape of Good Hope     45.790974          156          9.440222
      Mindoro Strait     41.636975           78          1.590657
      Oresund Strait     37.042755           85          5.853523
    Strait of Hormuz     36.897862           63         14.812747
          Suez Canal     

In [71]:
# Load disruption events
print("\nLoading disruption events...")
disrupt_file = "portwatch_disruptions_database_-3602226124776604501.csv"
df_disrupt = cudf.read_csv(disrupt_file)

print(f"✅ Loaded {len(df_disrupt):,} disruption events")

disrupt_pd = df_disrupt.to_pandas() if GPU_AVAILABLE else df_disrupt
print("\n⚠️ EVENT TYPES:")
print(disrupt_pd['eventtype'].value_counts())

print("\n🚨 ALERT LEVELS:")
print(disrupt_pd['alertlevel'].value_counts())


Loading disruption events...
✅ Loaded 125 disruption events

⚠️ EVENT TYPES:
eventtype
TC    70
EQ    29
FL    14
DR     4
OT     4
VO     2
WF     2
Name: count, dtype: int64

🚨 ALERT LEVELS:
alertlevel
RED       124
ORANGE      1
Name: count, dtype: int64


---
## 7. 💾 Save Processed Data

In [72]:
# Clean and save
print("Saving processed data...")

# Remove rows with NaN targets
df_clean = df.dropna(subset=['target_calls_1d'])
print(f"Clean records: {len(df_clean):,}")

# Save as parquet (GPU-accelerated write)
t0 = time.time()

# Full dataset
df_clean.to_parquet(OUTPUT_DIR / "gpu_port_features.parquet", index=False)
print(f"✅ Saved: gpu_port_features.parquet ({time.time()-t0:.3f}s)")

# Save surge analysis
surge_stats_pd.to_csv(OUTPUT_DIR / "gpu_surge_analysis.csv", index=False)
print(f"✅ Saved: gpu_surge_analysis.csv")

# Save chokepoints
df_choke.to_parquet(OUTPUT_DIR / "gpu_chokepoints.parquet", index=False)
print(f"✅ Saved: gpu_chokepoints.parquet")

Saving processed data...
Clean records: 75,690
✅ Saved: gpu_port_features.parquet (0.101s)
✅ Saved: gpu_surge_analysis.csv
✅ Saved: gpu_chokepoints.parquet


In [73]:
# Feature summary
feature_cols = [c for c in df_clean.columns if not c.startswith('target_')
                and c not in ['date', 'portname', 'portid', 'country', 'ISO3', 'ObjectId', 'port_tier']]
target_cols = [c for c in df_clean.columns if c.startswith('target_')]

feature_info = {
    'features': feature_cols,
    'targets': target_cols,
    'n_features': len(feature_cols),
    'n_targets': len(target_cols),
    'n_records': len(df_clean),
    'n_ports': int(df_clean['portname'].nunique()),
    'gpu_processed': GPU_AVAILABLE
}

with open(OUTPUT_DIR / "gpu_feature_info.json", 'w') as f:
    json.dump(feature_info, f, indent=2)

print(f"\n📊 FEATURE SUMMARY:")
print(f"  Features: {len(feature_cols)}")
print(f"  Targets: {len(target_cols)}")
print(f"  Records: {len(df_clean):,}")
print(f"  Ports: {df_clean['portname'].nunique()}")


📊 FEATURE SUMMARY:
  Features: 64
  Targets: 5
  Records: 75,690
  Ports: 30


---
## 8. 🤖 Quick ML Baseline (GPU with cuML)

In [74]:
# Try cuML for GPU-accelerated ML
try:
    from cuml.ensemble import RandomForestRegressor as cuRF
    from cuml.model_selection import train_test_split as cu_split
    from cuml.metrics import mean_absolute_error as cu_mae, r2_score as cu_r2
    CUML_AVAILABLE = True
    print("✅ cuML available for GPU-accelerated ML!")
except ImportError:
    from sklearn.ensemble import RandomForestRegressor as cuRF
    from sklearn.model_selection import train_test_split as cu_split
    from sklearn.metrics import mean_absolute_error as cu_mae, r2_score as cu_r2
    CUML_AVAILABLE = False
    print("⚠️ cuML not available, using sklearn")

✅ cuML available for GPU-accelerated ML!


In [75]:
# Prepare data for ML
print("Preparing ML data...")

# Get numeric features only
numeric_cols = df_clean.select_dtypes(include=['number']).columns.tolist()
ml_features = [c for c in numeric_cols if not c.startswith('target_')
               and c not in ['ObjectId', 'year']]

print(f"ML Features: {len(ml_features)}")

# Sample for faster training
n_samples = min(50000, len(df_clean))
if GPU_AVAILABLE:
    df_sample = df_clean.sample(n=n_samples, random_state=42)
    X = df_sample[ml_features].fillna(0)
    y = df_sample['target_calls_1d'].fillna(0)

    # Replace inf
    X = X.replace([cp.inf, -cp.inf], 0)
else:
    df_sample = df_clean.sample(n=n_samples, random_state=42)
    X = df_sample[ml_features].fillna(0).replace([np.inf, -np.inf], 0)
    y = df_sample['target_calls_1d'].fillna(0)

print(f"Training samples: {len(X):,}")

Preparing ML data...
ML Features: 63
Training samples: 50,000


In [76]:
# Train/Test split
X_train, X_test, y_train, y_test = cu_split(X, y, test_size=0.2, random_state=42)

print(f"Train: {len(X_train):,}, Test: {len(X_test):,}")

Train: 40,000, Test: 10,000


In [77]:
# Train Random Forest
print("Training Random Forest...")
t0 = time.time()

if CUML_AVAILABLE:
    model = cuRF(n_estimators=100, max_depth=12, random_state=42)
else:
    model = cuRF(n_estimators=100, max_depth=12, n_jobs=-1, random_state=42)

model.fit(X_train, y_train)
train_time = time.time() - t0

print(f"⏱️ Training time: {train_time:.2f}s")

Training Random Forest...
⏱️ Training time: 0.58s


In [78]:
# Evaluate
print("Evaluating model...")

y_pred = model.predict(X_test)

if CUML_AVAILABLE:
    mae = cu_mae(y_test, y_pred)
    r2 = cu_r2(y_test, y_pred)
else:
    mae = cu_mae(y_test, y_pred)
    r2 = cu_r2(y_test, y_pred)

print(f"\n🎯 MODEL PERFORMANCE:")
print(f"  MAE: {mae:.2f} port calls")
print(f"  R²: {r2:.3f} ({r2*100:.1f}% variance explained)")

Evaluating model...

🎯 MODEL PERFORMANCE:
  MAE: 1.49 port calls
  R²: 0.785 (78.5% variance explained)


In [79]:
# Feature importance
if hasattr(model, 'feature_importances_'):
    importance = model.feature_importances_
    if GPU_AVAILABLE and hasattr(importance, 'to_numpy'):
        importance = importance.to_numpy()
    elif GPU_AVAILABLE and hasattr(importance, 'get'):
        importance = importance.get()

    feat_imp = pd.DataFrame({
        'feature': ml_features,
        'importance': importance
    }).sort_values('importance', ascending=False)

    print("\n📊 TOP 20 IMPORTANT FEATURES:")
    print(feat_imp.head(20).to_string(index=False))

    # Save
    feat_imp.to_csv(OUTPUT_DIR / "gpu_feature_importance.csv", index=False)
    print(f"\n✅ Saved: gpu_feature_importance.csv")

---
## 9. 📋 Summary

In [80]:
# Final summary
print("="*80)
print("🎉 GPU EDA COMPLETE!")
print("="*80)

print(f"\n📊 DATA PROCESSED:")
print(f"  • Port records: {len(df_clean):,}")
print(f"  • US ports analyzed: {df_clean['portname'].nunique()}")
print(f"  • Chokepoint records: {len(df_choke):,}")
print(f"  • Disruption events: {len(df_disrupt):,}")

print(f"\n🔧 FEATURES ENGINEERED:")
print(f"  • Total features: {len(feature_cols)}")
print(f"  • Prediction targets: {len(target_cols)}")

print(f"\n🚨 SURGE ANALYSIS:")
print(f"  • Total surge events (2σ): {total_surge_2std:,}")
print(f"  • Average surge rate: {avg_surge_rate:.2f}%")

print(f"\n🤖 BASELINE MODEL:")
print(f"  • MAE: {mae:.2f} port calls")
print(f"  • R²: {r2:.3f}")

print(f"\n💾 OUTPUT FILES:")
for f in sorted(OUTPUT_DIR.glob("gpu_*")):
    size = f.stat().st_size / 1024
    unit = "KB"
    if size > 1024:
        size /= 1024
        unit = "MB"
    print(f"  • {f.name}: {size:.1f} {unit}")

print(f"\n✅ GPU Accelerated: {GPU_AVAILABLE}")
print(f"✅ cuML Used: {CUML_AVAILABLE}")

🎉 GPU EDA COMPLETE!

📊 DATA PROCESSED:
  • Port records: 75,690
  • US ports analyzed: 30
  • Chokepoint records: 70,728
  • Disruption events: 125

🔧 FEATURES ENGINEERED:
  • Total features: 64
  • Prediction targets: 5

🚨 SURGE ANALYSIS:
  • Total surge events (2σ): 523
  • Average surge rate: 0.69%

🤖 BASELINE MODEL:
  • MAE: 1.49 port calls
  • R²: 0.785

💾 OUTPUT FILES:
  • gpu_chokepoints.parquet: 4.0 MB
  • gpu_feature_info.json: 1.4 KB
  • gpu_port_features.parquet: 11.4 MB
  • gpu_surge_analysis.csv: 3.7 KB

✅ GPU Accelerated: True
✅ cuML Used: True


---
## 🚀 Next Steps

1. **Deep Learning Models**: Use the features to train LSTM/Transformer for time series forecasting
2. **Multi-port correlation**: Analyze how surges propagate between ports
3. **Chokepoint impact**: Link chokepoint congestion to US port arrivals
4. **Rail terminal prediction**: Estimate containers heading to rail based on import_container